In [1]:
import requests


In [2]:
# retrieve download link
result = requests.post('https://europe-west3-lumpito.cloudfunctions.net/download_files', json={
    "company": "Alasco",
    "company_id": "HRB 205171",
    "documents": ["gs", "si"]
})

In [4]:
result

JSONDecodeError: Extra data: line 1 column 5 (char 4)

In [23]:
response = result.json()

# filter python list for document url with type gs
gs_doc_url = list(filter(lambda x: x["type"] == "gs", response["document_urls"]))[0]["url"]

# download document
gs_doc = requests.get(gs_doc_url)

# store document as pdf file in folder output/
with open(f"output/gs-{response['name']}.pdf", "wb") as f:
    f.write(gs_doc.content)

JSONDecodeError: Extra data: line 1 column 5 (char 4)

In [ ]:
# load csv list into list of dictionaries
import csv

with open('startupdetector_founders_v1_MF - Startups.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    startup_list = list(reader)

print(startup_list[0])

startup_list[0]["Company_Name"]
startup_list[0]["Aktenzeichen"]



{'#': '1', 'Company_Name': 'BCN Food Tech GmbH', 'Aktenzeichen': 'HRB 272187', 'Date_Incorporation': '2021-12-31', 'CMP_Status': 'A', 'Amtsgericht': 'München', 'Street': 'Pestalozzistraße 34', 'Postcode': 'D-80469', 'City': 'München', 'Industry': 'Lebensmittel', 'WZ2008': '10', 'Startup_Website': '–', 'LinkedIn_URL': '–', 'Crunchbase_URL': '–', 'Northdata_URL': 'https://www.northdata.com/BCN+Food+Tech+GmbH,+M%C3%BCnchen/HRB+272187', 'Comment': '', 'Founding_Team_Size': '1'}


In [26]:
!pip install tqdm

  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [31]:
from tqdm import tqdm

In [51]:
import os
failed_docs = []

# ignore previously retrieved docs in output folder
retrieved_docs = os.listdir("output/gs")
retrieved_companies = [x.replace(".pdf", "") for x in retrieved_docs]

for i in tqdm(range(0,100)):
    if startup_list[i]["Company_Name"] in retrieved_companies:
        continue

    print(startup_list[i]["Company_Name"])

    try:
    # retrieve download link
        result = requests.post('https://europe-west3-lumpito.cloudfunctions.net/download_files', json={
            "company": startup_list[i]["Company_Name"],
            # "company_id": startup_list[i]["Aktenzeichen"],
            "company_id": None,
            "documents": ["gs", "si"]
        })

        response = result.json()

        # filter python list for document url with type gs
        gs_doc_url = list(filter(lambda x: x["type"] == "gs", response["document_urls"]))[0]["url"]

        # download document
        gs_doc = requests.get(gs_doc_url)

        # store document as pdf file in folder output/
        with open(f"output/gs/{response['name']}.pdf", "wb") as f:
            f.write(gs_doc.content)
    except Exception as e:
        failed_doc = {"company": startup_list[i]["Company_Name"], "company_id": startup_list[i]["Aktenzeichen"], "error": e }
        failed_docs.append(failed_doc)


  0%|          | 0/100 [00:00<?, ?it/s]

APFL Advanced Pet Food Labs UG (haftungsbeschränkt)


  4%|▍         | 4/100 [00:14<05:40,  3.55s/it]

AT - Alpi Tech UG (haftungsbeschränkt)


  5%|▌         | 5/100 [00:17<05:26,  3.43s/it]

...

 97%|█████████▋| 97/100 [05:53<00:07,  2.48s/it]

CapiWise UG (haftungsbeschränkt)


 98%|█████████▊| 98/100 [06:09<00:07,  3.52s/it]

HappieHaus Wellbeing GmbH


100%|██████████| 100/100 [06:26<00:00,  3.87s/it]


In [56]:
failed_docs

# # filter where error does not contain 'Extra data'
# failed_docs_uncommon = list(filter(lambda x: 'Extra data' not in x["error"], failed_docs))


# failed_docs_uncommon

[{'company': 'AT - Alpi Tech UG (haftungsbeschränkt)',
  'company_id': 'HRB 272118',
  'error': requests.exceptions.JSONDecodeError('Extra data: line 1 column 5 (char 4)')},
 {'company': "Htwo Power 'n' Heat GmbH",
  'company_id': 'HRB 36196 P',
  'error': requests.exceptions.ConnectionError(urllib3.exceptions.ProtocolError('Connection aborted.',
                                                                       http.client.RemoteDisconnected('Remote end closed connection without response')))},
 {'company': 'REnergi UG (haftungsbeschränkt)',
  'company_id': 'HRB 25421',
  'error': requests.exceptions.JSONDecodeError('Extra data: line 1 column 5 (char 4)')},
 {'company': 'Revalu Impact AG',
  'company_id': 'HRB 272158',
  'error': requests.exceptions.JSONDecodeError('Extra data: line 1 column 5 (char 4)')},
 {'company': 'SeeMe HealthCare Technologies UG (haftungsbeschränkt)',
  'company_id': 'HRB 272116',
  'error': requests.exceptions.JSONDecodeError('Extra data: line 1 column 5 (c